# Defining Data Structures with SQL Syntax

This notebook is designed for users who prefer to define data structures using SQL syntax. It guides you through the process of taking SQL table creation statements and query specifications and generating a corresponding structured relational database schema using DataWizzAI.

# Initial Setup Guide

## Import Required Packages

In [28]:
# First, import all the necessary packages.
from langchain_openai import ChatOpenAI
from src.DataDefiner import *
from src.DataAugmentor import DataAugmentor
from src.utils.utils import parse_output, try_parse_json, create_json_sample_from_csv, compose_query_message


In [29]:
## Load Environment Variables

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


## Initialize the Language Model

In [30]:
# Please make sure OPENAI_API_KEY is loaded to your environment variables
# Initialize language model
llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo")


# Defining a Data Structure - SQLToTabular

In [31]:
# Initialize a DataStructureDefiner for the task of defining a relational DB with sql commands
pipeline_name = get_pipeline_name('SQLToTabular')
DataDefinerObj = DataDefiner(llm, pipeline_name=pipeline_name)

In [32]:
# Define the required data structure and view the result structure
dataStructureDescription = \
"-- Create a 'Customers' table with customer information\
CREATE TABLE Customers (\
    CustomerID INT PRIMARY KEY,\
    FirstName VARCHAR(50),\
    LastName VARCHAR(50),\
    Email VARCHAR(100),\
    JoinDate DATE\
);\
\
-- Create an 'Orders' table with order details\
CREATE TABLE Orders (\
    OrderID INT PRIMARY KEY,\
    CustomerID INT,\
    OrderDate DATE,\
    ProductCategory VARCHAR(50),\
    ProductName VARCHAR(100),\
    Units INT,\
    TotalAmount DECIMAL(10, 2),\
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)\
);"

In [33]:
# Convert the textual description into a sample of the needed data
dataStructureSample = DataDefinerObj.define_schema_from_description(description=dataStructureDescription)
print(parse_output(dataStructureSample))



> Entering new LLMChain chain...
Prompt after formatting:
You are a system that specializes in generating synthetic data according to user requests.Generate an initial synthetic data sample for the described task (according to the user guidance if provided, or your knowledge otherwise).Provide a sample with 10 number of items; items can be records in a table,records that can potentially be joined in a DB schema, or the equivalent tuples in a JSON format in unstructured text.The task as described by the user: -- Create a 'Customers' table with customer informationCREATE TABLE Customers (    CustomerID INT PRIMARY KEY,    FirstName VARCHAR(50),    LastName VARCHAR(50),    Email VARCHAR(100),    JoinDate DATE);-- Create an 'Orders' table with order detailsCREATE TABLE Orders (    OrderID INT PRIMARY KEY,    CustomerID INT,    OrderDate DATE,    ProductCategory VARCHAR(50),    ProductName VARCHAR(100),    Units INT,    TotalAmount DECIMAL(10, 2),    FOREIGN KEY (CustomerID) REFERENCES Cu

## Optional: create a TaskSpecificationAugmentor

You can augment your data description and turn it into a detailed data requirements specifications by using the TaskSpecificationAugmentor object. This component imitates a data analyst that learns your requirement and translate it into a detailed specification of the needed data and its characteristics. 

In [34]:
# Generate professional specifications
task_specifications = TaskSpecificationAugmentor.generate_specifications_from_description(llm, description=dataStructureDescription)

# Print the generated specifications
print("Generated Specifications:")
print(task_specifications)



> Entering new LLMChain chain...
Prompt after formatting:
You are an analyst whose job is to conduct a deep research for a given task, and define the needed data to collect, making sure you don't miss any relevant detail, and gain a deep understanding of the data characteristics.1.  Generate the names of the columns relevant for the description of the user. Use indicative names. Assign each column its type - numerical, categorical, datetime, free text, unique identifier. 2. Revisit each column and complete these details:For numeric columns - describe its distribution, mean and std, min and max values. for numbers and datetimes define the needed format, for categorical columns detail a complete set of categories and its probabilities. For free text columns - specify the mean and std of the text length, For unique identifier columns - specify the format and regEX to follow. For Datetime columns - specify min and max values, as well as the time intervals mean and std values. 3. Importan

In [35]:
# Convert the textual description, aided with the expert specification, into a sample of the needed data
dataStructureSample = DataDefinerObj.define_schema_from_description(description=dataStructureDescription,
                                                                    task_specifications=task_specifications)
print(parse_output(dataStructureSample))



> Entering new LLMChain chain...
Prompt after formatting:
You are a system that specializes in generating synthetic data according to user requests.Generate an initial synthetic data sample for the described task (according to the user guidance if provided, or your knowledge otherwise).Provide a sample with 10 number of items; items can be records in a table,records that can potentially be joined in a DB schema, or the equivalent tuples in a JSON format in unstructured text.The task as described by the user: -- Create a 'Customers' table with customer informationCREATE TABLE Customers (    CustomerID INT PRIMARY KEY,    FirstName VARCHAR(50),    LastName VARCHAR(50),    Email VARCHAR(100),    JoinDate DATE);-- Create an 'Orders' table with order detailsCREATE TABLE Orders (    OrderID INT PRIMARY KEY,    CustomerID INT,    OrderDate DATE,    ProductCategory VARCHAR(50),    ProductName VARCHAR(100),    Units INT,    TotalAmount DECIMAL(10, 2),    FOREIGN KEY (CustomerID) REFERENCES Cu

# Generating Data

In [36]:
# First initialize the DataAugmentor object with your chosen language model (llm) and the predefined data structure (dataStructureSample):
DataAugmentorObj = DataAugmentor(llm=llm, structure=dataStructureSample)

## Generate a sample (for output validation)

In [37]:
# You can view a sample of the generated data:
generated_data = DataAugmentorObj.preview_output_sample()
print(generated_data)



> Entering new LLMChain chain...
Prompt after formatting:
You are a system that specializes in generating synthetic data according to user requests.Generate 10 Sample synthetic items(records in a table or the equivalent elements in unstructured text) for the data described below. In the following structure, but with different values.For the following task: Data description: None.Required Structure: {
  "Customers": [
    {
      "CustomerID": 1,
      "FirstName": "John",
      "LastName": "Doe",
      "Email": "johndoe@email.com",
      "JoinDate": "2022-01-01"
    },
    {
      "CustomerID": 2,
      "FirstName": "Jane",
      "LastName": "Smith",
      "Email": "janesmith@email.com",
      "JoinDate": "2022-02-15"
    },
    {
      "CustomerID": 3,
      "FirstName": "Michael",
      "LastName": "Johnson",
      "Email": "michaeljohnson@email.com",
      "JoinDate": "2022-03-10"
    },
    {
      "CustomerID": 4,
      "FirstName": "Sarah",
      "LastName": "Williams",
      "

## Optional: query/filter the data structure to control the generated content

In [38]:
# You can also add queries and filters to guide the generated contents:
language = 'English'
query = "Only sports product category are included "

generated_data = DataAugmentorObj.preview_output_sample(query=query, language=language)
print(generated_data)



> Entering new LLMChain chain...
Prompt after formatting:
You are a system that specializes in generating synthetic data according to user requests.Generate 10 Sample synthetic items(records in a table or the equivalent elements in unstructured text) for the data described below. In the following structure, but with different values.For the following task: Data description: Only sports product category are included  ; All texts should be translated to English language..Required Structure: {
  "Customers": [
    {
      "CustomerID": 1,
      "FirstName": "John",
      "LastName": "Doe",
      "Email": "johndoe@email.com",
      "JoinDate": "2022-01-01"
    },
    {
      "CustomerID": 2,
      "FirstName": "Jane",
      "LastName": "Smith",
      "Email": "janesmith@email.com",
      "JoinDate": "2022-02-15"
    },
    {
      "CustomerID": 3,
      "FirstName": "Michael",
      "LastName": "Johnson",
      "Email": "michaeljohnson@email.com",
      "JoinDate": "2022-03-10"
    },
  

In [39]:
# If you used TaskSpecificationAugmentor for defining this structure, and you wish to add queries and filters to guide the content generate, it will be a good idea to revisit the expert specifications to adjust it to the user query and guidance while maintaining the external wisdom:
full_query = compose_query_message(query=query, language=language)
updated_task_specifications = TaskSpecificationAugmentor.refine_specifications_by_description(llm=llm,
description=full_query,previous_task_specification=task_specifications)

generated_data = DataAugmentorObj.preview_output_sample(query=query, language=language,
                                                        task_specifications=updated_task_specifications)
print(generated_data)



> Entering new LLMChain chain...
Prompt after formatting:
You are an analyst whose job is to conduct a deep research for a given task, and define the needed data to collect, making sure you don't miss any relevant detail, and gain a deep understanding of the data characteristics.You already gave instructions for the needed data (see Previous Task Specifications), but now the user asks a content refinement (see User Query). Please revisit the columns distributions, and descriptive statistics and update those that have changed due to the user query. 
The User Query: Data description: Only sports product category are included  ; All texts should be translated to English language.;
Your Previous Task Specifications: 1. Customers Table:
- CustomerID (numerical, unique identifier)
- FirstName (categorical, free text)
- LastName (categorical, free text)
- Email (categorical, free text)
- JoinDate (datetime)

2. Customers Table Details:
- CustomerID: Unique identifier, with distribution deta

## Generating Full Output

To generate the full dataset, use the generate_data method. Specify your query (if any), optionaly the region and language, and the number of records you wish to generate. 

In [40]:
# Without expert specifications
generated_data = DataAugmentorObj.generate_data( query=query, language=language, num_records=15) 

# With expert specifications
#generated_data = DataAugmentorObj.generate_data( query=query, language=language, task_specifications=updated_task_specifications, num_records=15) 

generated_data



> Entering new LLMChain chain...
Prompt after formatting:
You are a system that specializes in generating synthetic data according to user requests.Generate 10 Sample synthetic items(records in a table or the equivalent elements in unstructured text) for the data described below. In the following structure, but with different values.For the following task: Data description: Only sports product category are included  ; All texts should be translated to English language..Required Structure: {
  "Customers": [
    {
      "CustomerID": 1,
      "FirstName": "John",
      "LastName": "Doe",
      "Email": "johndoe@email.com",
      "JoinDate": "2022-01-01"
    },
    {
      "CustomerID": 2,
      "FirstName": "Jane",
      "LastName": "Smith",
      "Email": "janesmith@email.com",
      "JoinDate": "2022-02-15"
    },
    {
      "CustomerID": 3,
      "FirstName": "Michael",
      "LastName": "Johnson",
      "Email": "michaeljohnson@email.com",
      "JoinDate": "2022-03-10"
    },
  

{'Customers':     CustomerID FirstName  LastName                    Email    JoinDate
 0           31    Oliver     Evans    oliverevans@email.com  2024-12-10
 1           32    Sophia     Perez    sophiaperez@email.com  2025-01-15
 2           33     Lucas   Ramirez   lucasramirez@email.com  2025-02-20
 3           34      Lily    Nguyen     lilynguyen@email.com  2025-03-25
 4           35     Henry  Gonzalez  henrygonzalez@email.com  2025-04-30
 5           41     Alice   Johnson   alicejohnson@email.com  2026-01-01
 6           42   William    Miller  williammiller@email.com  2026-02-15
 7           43    Sophie     Brown    sophiebrown@email.com  2026-03-10
 8           44     Mason    Garcia    masongarcia@email.com  2026-04-05
 9           45     Chloe  Martinez  chloemartinez@email.com  2026-05-20
 10          46    Oliver    Wilson   oliverwilson@email.com  2026-06-01
 11          47      Ella  Anderson   ellaanderson@email.com  2026-07-15
 12          48     Henry  Thompson  h

## Generating Full Output in Parallel

For more efficient data generation, especially when dealing with large datasets or multiple requests, our package supports parallel processing. This section covers how to utilize the generate_data_in_parallel method of the DataAugmentor class to generate your dataset asynchronously.


### Setup for Parallel Execution

To ensure smooth parallel execution, especially within environments that don't natively support asynchronous operations (like Jupyter notebooks), we use nest_asyncio. This module allows asyncio to run inside environments with their own event loops.

In [41]:
import nest_asyncio
nest_asyncio.apply()

### Generate Full Output in Parallel

To generate data in parallel, use the generate_data_in_parallel coroutine. This method allows you to specify the query (if any), the number of records, region, and language, similarly to generate_data, but executes multiple data generation tasks concurrently.

In [42]:
import asyncio
# Without expert specifications
generated_data = asyncio.run(DataAugmentorObj.generate_data_in_parallel(query = "", records=20, language=language))

generated_data

# With expert specifications
#generated_data = asyncio.run(DataAugmentorObj.generate_data_in_parallel(query = "", records=20, task_specifications=updated_task_specifications, language=language))



> Entering new LLMChain chain...

> Entering new LLMChain chain...

Prompt after formatting:
You are a system that specializes in generating synthetic data according to user requests.Generate 10 Sample synthetic items(records in a table or the equivalent elements in unstructured text) for the data described below. In the following structure, but with different values.For the following task: Data description:  ; The required region: US ; All texts should be translated to English language..Required Structure: {
  "Customers": [
    {
      "CustomerID": 1,
      "FirstName": "John",
      "LastName": "Doe",
      "Email": "johndoe@email.com",
      "JoinDate": "2022-01-01"
    },
    {
      "CustomerID": 2,
      "FirstName": "Jane",
      "LastName": "Smith",
      "Email": "janesmith@email.com",
      "JoinDate": "2022-02-15"
    },
    {
      "CustomerID": 3,
      "FirstName": "Michael",
      "LastName": "Johnson",
      "Email": "michaeljohnson@email.com",
      "JoinDate": "20

{'Customers':    CustomerID    FirstName   LastName                          Email  \
 0           6       Amanda     Taylor         amandataylor@email.com   
 1           7  Christopher   Anderson  christopheranderson@email.com   
 2           8       Olivia   Martinez       oliviamartinez@email.com   
 3           9      Matthew     Garcia        matthewgarcia@email.com   
 4          10       Lauren  Rodriguez      laurenrodriguez@email.com   
 5           6        Emily      Jones           emilyjones@email.com   
 6           7      Matthew      Brown         matthewbrown@email.com   
 7           8       Olivia      Davis          oliviadavis@email.com   
 8           9        Ethan     Wilson          ethanwilson@email.com   
 9          10          Ava   Martinez          avamartinez@email.com   
 
      JoinDate  
 0  2022-06-01  
 1  2022-07-15  
 2  2022-08-10  
 3  2022-09-05  
 4  2022-10-20  
 5  2022-06-01  
 6  2022-07-15  
 7  2022-08-10  
 8  2022-09-05  
 9  2022-10-